In [1]:
import pandas as pd
import numpy as np
import ast
import math
import networkx as nx
from pyvis.network import Network
import plotly.graph_objects as go
import sys
sys.path.append('../')
from modules import utils, constants

#### The data

In [2]:
test_df = pd.read_csv('../test_dfs/negative_reward/step_reward_eighteenth_100m.csv').drop(['index'], axis=1)
test_df.head()

,episode_length,reward,y_pred,y_actual,trajectory,terminated,is_success
0,4.0,0.833333,1.0,1.0,"['ana', 'pericardial_effusion', 'cutaneous_lup...",0.0,1.0
1,13.0,0.333333,1.0,1.0,"['ana', 'pericardial_effusion', 'cutaneous_lup...",0.0,1.0
2,7.0,0.666667,1.0,1.0,"['ana', 'pericardial_effusion', 'cutaneous_lup...",0.0,1.0
3,14.0,0.277778,1.0,1.0,"['ana', 'pericardial_effusion', 'cutaneous_lup...",0.0,1.0
4,7.0,0.666667,1.0,1.0,"['ana', 'pericardial_effusion', 'cutaneous_lup...",0.0,1.0


In [3]:
utils.success_rate(test_df)[0]

93.25714285714287

In [4]:
testing_df = pd.read_csv('../data/missingness/0/testing_set.csv')
testing_df.head()

,ana,fever,leukopenia,thrombocytopenia,auto_immune_hemolysis,delirium,psychosis,seizure,non_scarring_alopecia,oral_ulcers,...,joint_involvement,proteinuria,anti_cardioliphin_antibodies,anti_β2gp1_antibodies,lupus_anti_coagulant,low_c3,low_c4,anti_dsdna_antibody,anti_smith_antibody,label
0,1,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
1,1,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,1
2,1,0,0,0,0,0,1,0,0,0,...,1,0,0,0,1,1,0,0,0,1
3,1,1,0,0,0,0,0,0,1,0,...,0,0,0,0,1,0,0,1,0,1
4,1,0,0,0,0,0,0,0,1,0,...,1,0,0,0,0,1,0,0,0,1


In [5]:
training_df = pd.read_csv('../data/missingness/0/training_set.csv')
training_df.head()

,ana,fever,leukopenia,thrombocytopenia,auto_immune_hemolysis,delirium,psychosis,seizure,non_scarring_alopecia,oral_ulcers,...,joint_involvement,proteinuria,anti_cardioliphin_antibodies,anti_β2gp1_antibodies,lupus_anti_coagulant,low_c3,low_c4,anti_dsdna_antibody,anti_smith_antibody,label
0,0,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1
3,1,0,0,0,0,0,0,1,1,0,...,0,1,0,0,0,0,0,1,1,1
4,1,0,0,1,1,1,0,0,0,0,...,0,1,0,0,0,1,0,0,1,1


#### Possibly useful functions

In [6]:
def generate_tuple_dict(df):
    frequency_dict = {}
    for traj in df.trajectory:
        if traj in frequency_dict.keys():
            frequency_dict[traj] += 1
        else:
            frequency_dict[traj] = 1
    overall_tup_dict = {}
    for key, value in frequency_dict.items():
        new_key = ast.literal_eval(key)
        for tup in zip(new_key, new_key[1:]):
            if tup in overall_tup_dict.keys():
                overall_tup_dict[tup] += value
            else:
                overall_tup_dict[tup] = value
    return overall_tup_dict

In [7]:
def create_sankey_df(df):
    overall_tup_dict = generate_tuple_dict(df)
    sankey_df = pd.DataFrame()
    sankey_df['source'] = [i[0] for i in overall_tup_dict.keys()]
    sankey_df['target'] = [i[1] for i in overall_tup_dict.keys()]
    sankey_df['value'] = list(overall_tup_dict.values())
    sankey_df['link_type'] = sankey_df['target'].apply(lambda i: 'terminal' if i in ['No lupus', 'Lupus', 'Inconclusive diagnosis'] else 'non_terminal')
    return sankey_df

In [8]:
def draw_pyvis_network(pathways_df, pathway_type='network'): #second type is trajectory 
    pathways_df['type'] = 'directed'
    start_node = pathways_df.iloc[0]['source']
    all_nodes = list(set(pathways_df.source.unique().tolist() + pathways_df.target.unique().tolist()))
    terminal_nodes = list(set(pathways_df[pathways_df.link_type=='terminal'].target))
    non_terminal_nodes = [i for i in all_nodes if i not in terminal_nodes and i!=start_node]
    
    #change get_net to something else since it satnds for Game of Thrones
    got_net = Network(notebook=True, height='600px', width='100%', directed=True, cdn_resources='in_line')
    got_net.add_node(start_node, color='purple', size=20)
    got_net.add_nodes(non_terminal_nodes, size=[15]*len(non_terminal_nodes), color=['blue']*len(non_terminal_nodes))
    got_net.add_nodes(terminal_nodes, color=['green']*len(terminal_nodes), size=[20]*len(terminal_nodes))
    for src, target, value, edge_thresh in zip(pathways_df.source, pathways_df.target, pathways_df.value, pathways_df.edge_threshold):
        if pathway_type =='network':
            if value > threshold:
                if math.isnan(edge_thresh):
                    got_net.add_edge(src, target, value=value, color='red')
                else:
                    got_net.add_edge(src, target, value=value, color='red', label=str(int(edge_thresh)))
            else:
                got_net.add_edge(src, target, value=value, color='blue')
        elif pathway_type =='trajectory': #It's just one trajectory
            if isinstance(edge_thresh, str):
                got_net.add_edge(src, target, value=value, color='blue', label=edge_thresh)
            else:
                got_net.add_edge(src, target, value=value, color='blue', label=str(int(edge_thresh)))
        else:
            print('Unknown pathway type')
    return got_net

#### start from here

In [9]:
def get_threshold_value(df, source): #source e.g. 'ana'
    threshold_values = df[source].unique().tolist()
    if len(threshold_values) == 1:
        return int(threshold_values[0])
    else:
        return str(threshold_values)

In [10]:
def draw_one_trajectory(pred_df, trajectory): #pred_df = pred_no_lupus par exemple
    trajectory_df = pred_df[pred_df.trajectory == trajectory]
    trajectory_testing_df = testing_df.loc[trajectory_df.index]
    trajectory_pathways_df = create_sankey_df(trajectory_df)
    trajectory_pathways_df['edge_threshold'] = [get_threshold_value(trajectory_testing_df, source) for source in trajectory_pathways_df.source]
    trajectory_net = draw_pyvis_network(trajectory_pathways_df, 'trajectory')   
    return trajectory_net

In [25]:
pred_no_lupus = test_df[test_df['y_pred']==0]
no_lupus_pathway_1 = pred_no_lupus.trajectory.value_counts().index[4]
no_lupus_pathway_1

"['ana', 'pericardial_effusion', 'cutaneous_lupus', 'fever', 'anti_smith_antibody', 'joint_involvement', 'low_c3', 'non_scarring_alopecia', 'proteinuria', 'pleural_effusion', 'anti_β2gp1_antibodies', 'low_c4', 'anti_dsdna_antibody', 'No lupus']"

In [26]:
pred_no_lupus.episode_length.unique()

array([14.,  6., 15., 18., 12., 11., 13., 16.,  8., 17.,  9., 19., 21.,
       20.])

In [27]:
no_lupus_pathway1_net = draw_one_trajectory(pred_no_lupus, no_lupus_pathway_1)
no_lupus_pathway1_net.show('Example.html')

In [36]:
pred_lupus = test_df[test_df['y_pred']==1]
lupus_pathway_0 = pred_lupus.trajectory.value_counts().index[4]
lupus_pathway_0

"['ana', 'pericardial_effusion', 'cutaneous_lupus', 'Lupus']"

In [37]:
lupus_pathway0_net = draw_one_trajectory(pred_lupus, lupus_pathway_0)
lupus_pathway0_net.show('Example.html')

In [38]:
longest_lupus_pathway = pred_lupus[pred_lupus.episode_length==pred_lupus.episode_length.max()].trajectory.value_counts().index[0]#.iloc[0]['trajectory']
longest_lupus_pathway

"['ana', 'pericardial_effusion', 'cutaneous_lupus', 'fever', 'anti_smith_antibody', 'low_c3', 'anti_β2gp1_antibodies', 'non_scarring_alopecia', 'low_c4', 'thrombocytopenia', 'proteinuria', 'anti_dsdna_antibody', 'joint_involvement', 'delirium', 'pleural_effusion', 'leukopenia', 'seizure', 'psychosis', 'Lupus']"

In [39]:
longest_lupus_pathway_net = draw_one_trajectory(pred_lupus, longest_lupus_pathway)
longest_lupus_pathway_net.show('Example.html')